# Preparing data for r plots

This file gerneates csv files that are used in the r files to generate plots

In [74]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import numpy as np

In [75]:
# this file is generated from `natinoal_local_prepare_data.ipynb`.
df_category_grouped_class = pd.read_csv("../data/house_analysis/domain_rank_ishome_category_counts.csv")

In [76]:
def check_is_news(col):
    if col == "news":
        return True
    else:
        return False

In [77]:
df_category_grouped_class['is_news'] = df_category_grouped_class['category'].apply(check_is_news)

In [78]:
df_category_grouped_class['counts'].sum()

299573378

In [79]:
def check_is_local(col):
    if pd.isna(col):
        return None
    elif col == "local":
        return True
    else:
        return False

In [80]:
df_category_grouped_class['is_local'] = df_category_grouped_class['classification'].apply(check_is_local)

In [81]:
def is_poli_controlled(row):
    if row['category']=='campaign_personal':
        return True
    elif row['category']=='gov':
        if row['domain']=='house.gov':
            return True
        else:
            return False
    elif row['category']=='social':
        if row['domain']=='youtube.com':
            return False
        else:
            return True
    else:
        return False

In [82]:
df_category_grouped_class['is_poli_control'] = df_category_grouped_class.apply(is_poli_controlled, axis=1)

In [84]:
df_category_grouped_class['rank'] = df_category_grouped_class['serp_rank'] +1

In [102]:
df_category_grouped_class

,domain,serp_rank,is_home_district,category,counts,classification,is_news,is_local,is_poli_control,rank
0,1011now.com,2.0,False,news,2,local,True,True,False,3.0
1,1011now.com,3.0,False,news,13,local,True,True,False,4.0
2,1011now.com,4.0,False,news,1200,local,True,True,False,5.0
3,1011now.com,5.0,False,news,6503,local,True,True,False,6.0
4,1011now.com,5.0,True,news,18,local,True,True,False,6.0
...,...,...,...,...,...,...,...,...,...,...
94153,zoelofgren.com,21.0,False,campaign_personal,446,NaN,False,None,True,22.0
94154,zoelofgren.com,21.0,True,campaign_personal,1,NaN,False,None,True,22.0
94155,zoelofgren.com,22.0,False,campaign_personal,27,NaN,False,None,True,23.0
94156,zoelofgren.com,23.0,False,campaign_personal,435,NaN,False,None,True,24.0


In [103]:
df_category_grouped_class.to_csv("../data/house_analysis/domain_isnews_islocal_ispolicontrol.csv", index=False)

# Additoinal Analysis

In [85]:
df_category_grouped = df_category_grouped_class.groupby(['category','serp_rank'], dropna=True, as_index=False)['counts'].sum()

In [86]:
df_category_grouped

,category,serp_rank,counts
0,campaign_personal,0.0,324470
1,campaign_personal,1.0,2370594
2,campaign_personal,2.0,3208090
3,campaign_personal,3.0,1818414
4,campaign_personal,4.0,924530
...,...,...,...
226,social,35.0,3049
227,social,36.0,869
228,social,37.0,397
229,social,38.0,13


In [87]:
df_category_grouped['rank'] = df_category_grouped['serp_rank']+1

In [88]:
categories = df_category_grouped['category'].unique()

In [89]:
for cat in categories:
    sub_df = df_category_grouped[df_category_grouped['category']==cat]
    weighted_avg = np.average(sub_df['rank'], weights=sub_df['counts'])
    print(cat, ":", weighted_avg)

campaign_personal : 8.388386396795859
education : 14.291480703596745
gov : 7.235590963899232
news : 11.217542848709236
other_third_party : 11.681472532377038
social : 9.782569277135416


In [90]:
df_poly_grouped = df_category_grouped_class.groupby(['is_poli_control','rank'], dropna=True, as_index=False)['counts'].sum()

In [91]:
categories = df_poly_grouped['is_poli_control'].unique()

In [92]:
for cat in categories:
    sub_df = df_poly_grouped[df_poly_grouped['is_poli_control']==cat]
    weighted_avg = np.average(sub_df['rank'], weights=sub_df['counts'])
    print(cat, ":", weighted_avg)

False : 12.224093374184198
True : 7.438858317029485


# Top domain in third party

In [93]:
thid_party = df_category_grouped_class[df_category_grouped_class['category']=='other_third_party']

In [94]:
thid_party_domains = thid_party.groupby(['domain'], dropna=False, as_index=False)['counts'].sum()

In [95]:
thid_party_domains['percent'] = thid_party_domains['counts']/thid_party_domains['counts'].sum()*100

In [96]:
thid_party_domains.sort_values('counts', ascending=False)

,domain,counts,percent
354,wikipedia.org,20361206,34.053138
40,ballotpedia.org,18272995,30.560706
153,govtrack.us,16055696,26.852380
247,opensecrets.org,916820,1.533337
98,cq.com,813229,1.360086
...,...,...,...
300,songhall.org,1,0.000002
89,commondefense.us,1,0.000002
221,moffitt.org,1,0.000002
178,iranianamericanpac.org,1,0.000002


# Proportion of each variable

In [97]:
variables = ["is_news","is_local","is_poli_control"]

In [98]:
total = df_category_grouped_class['counts'].sum()

In [99]:
is_poli_control_grouped = df_category_grouped_class.groupby(['is_poli_control'], dropna=False, as_index=False)['counts'].sum()

In [100]:
is_poli_control_grouped['rate'] = is_poli_control_grouped['counts']/total

In [101]:
is_poli_control_grouped

,is_poli_control,counts,rate
0,False,159172475,0.531331
1,True,140400903,0.468669
